# Part 2.2. ReLU


In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

## 1. Problem of Sigmoid
sigmoid를 활성화 함수로 사용할 때 기울기를 계산할 때 `파란색` 부분은 기울기가 잘 계산되지만 `빨간색` 부분은 기울기가 거의 0에 수렴에 0 혹은 0에 아주 가까운 값이 기울기가 된다.

![9-1.png](./img/9-1.png)

기울기 계산 즉, backpropagation을 할 때 기울기를 앞의 노드로 전달시킬 때 활성화 함수에서의 기울기를 곱하게 되는데 아주 작은 값이 곱해지면서 더더 앞으로 갈수록 기울기가 소멸되는 현상이 발생하게 된다. 1층 신경망은 괜찮을 수 있지만 여러 개의 깊이가 있는 신경망에서는 위와 같은 현상이 발생할 수 있다.

![9-2.png](./img/9-2.png)

## 2. ReLU
$f(x) = max(0, x)$로 0보다 크면 기울기가 1, 0보다 작으면 0인 활성화 함수이다. 다만 활성화 함수 값이 0보다 작아지면 기울기가 소멸되는 단점이 있다.

## 3. Optimizer in Pytorch
`torch.optim`은 다양한 optimization 알고리즘을 갖고 있는 라이브러리이다.

![9-4.png](./img/9-4.png)

## 4. MNIST Example

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


In [0]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [0]:
# model construction
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [8]:
# Initialization with normal distribution
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [0]:
# model
# 마지막 활성화 함수로 softmax를 사용 <- CEE 때문에
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [0]:
# cost function and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# training
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 129.319229126
Epoch: 0002 cost = 36.202365875
Epoch: 0003 cost = 23.048833847
Epoch: 0004 cost = 16.014511108
Epoch: 0005 cost = 11.559041023
Epoch: 0006 cost = 8.540482521
Epoch: 0007 cost = 6.442119598
Epoch: 0008 cost = 4.754306316
Epoch: 0009 cost = 3.617706537
Epoch: 0010 cost = 2.721715212
Epoch: 0011 cost = 2.087671995
Epoch: 0012 cost = 1.720332623
Epoch: 0013 cost = 1.201963425
Epoch: 0014 cost = 0.999537766
Epoch: 0015 cost = 0.778918564
Learning finished


In [12]:
# test
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9483999609947205


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
